<a href="https://colab.research.google.com/github/Devesh946/AI_df20555/blob/main/Lab8/Lab8_data_feature_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!wget https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip

--2021-05-05 00:18:27--  https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20194537 (19M) [application/zip]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>]  19.26M  49.7MB/s    in 0.4s    

2021-05-05 00:18:27 (49.7 MB/s) - ‘data.zip.1’ saved [20194537/20194537]



In [18]:
!unzip data.zip

Archive:  data.zip
replace data/cats/cat.1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [23]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [24]:
print(len(os.listdir('./data/Humans/')))
print(len(os.listdir('./data/cats/')))
print(len(os.listdir('./data/dogs/')))
print(len(os.listdir('./data/horses/')))

202
202
202
202


In [25]:
path_humans = './data/Humans'
path_cats = './data/cats'
path_dogs = './data/dogs/'
path_horses = './data/horses/'

humans = os.listdir(path_humans)
cats = os.listdir(path_cats)
dogs = os.listdir(path_dogs)
horses = os.listdir(path_horses)

human_image_sizes = []
cats_image_sizes = []
dogs_image_sizes = []
horses_image_sizes = []

for i in range(202):
  hu = plt.imread(os.path.join(path_humans, humans[i]))
  c = plt.imread(os.path.join(path_cats, cats[i]))
  d = plt.imread(os.path.join(path_dogs, dogs[i]))
  ho = plt.imread(os.path.join(path_horses, horses[i]))
  human_image_sizes.append(hu.shape[:-1])
  cats_image_sizes.append(c.shape[:-1])
  dogs_image_sizes.append(d.shape[:-1])
  horses_image_sizes.append(ho.shape[:-1])

In [26]:
mean_sizes = [np.mean(human_image_sizes, axis=0), np.mean(cats_image_sizes, axis=0),
              np.mean(dogs_image_sizes, axis=0), np.mean(horses_image_sizes, axis=0)]

target_size = np.array(mean_sizes).mean(axis=0)
target_size

array([304.6200495 , 329.92574257])

In [27]:
image_generator = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

In [28]:
train_data = image_generator.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')

val_data = image_generator.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

Found 728 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [29]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

image_size = [304,329,3]

In [30]:
input = Input(shape = image_size)
model = Xception(input_tensor=input,)

91889664/91884032 [==============================] - 1s 0us/step


In [31]:
model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 151, 164, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 151, 164, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 151, 164, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [32]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
  layer.trainable = False


model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 151, 164, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 151, 164, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 151, 164, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")

In [34]:
my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [35]:
model.fit(train_data, validation_data=val_data, epochs=50, callbacks=my_callbacks)

Epoch 1/50
182/182 [==============================] - 355s 2s/step - loss: 0.6914 - accuracy: 0.7592 - val_loss: 0.0817 - val_accuracy: 0.9875
Epoch 2/50
 49/182 [=======>......................] - ETA: 3:56 - loss: 0.1394 - accuracy: 0.9585

KeyboardInterrupt: ignored

In [36]:
model.history.history

{'accuracy': [0.901098906993866],
 'loss': [0.37938669323921204],
 'val_accuracy': [0.987500011920929],
 'val_loss': [0.08166465908288956]}